In [1]:
from pywinauto import Desktop

# List all visible windows and their titles
for w in Desktop(backend="win32").windows():
    print(w.window_text())


ATKOSD2
GDI+ Window (AsusOSD.exe)
Task Switching








auto_mnova_pred_2.ipynb - nmr - Visual Studio Code
13C Prediction
1H Prediction
MestReNova

Documents - File Explorer
GDI+ Window (Explorer.EXE)
Progress



AMD Software: Adrenalin Edition
RadeonSoftware
RadeonSoftware
RadeonSoftware










C:\Users\bikas\AppData\Local\Programs\Ollama\ollama.exe

QTrayIconMessageWindow


bluestacks-services
AMD:CCC-AEMCapturingWindow







BackgroundModeTrayIconClass




Settings
RealtekAudioBackgroundProcessClass

DDE Server Window



OneDrive - Personal



SecurityHealthSystray
ATKOSD2 MainFrm Window (Help)



DesktopWindowXamlSource
GameVisual User Hidden Wnd
.NET-BroadcastEventWindow.21af1a5.0

ARMOURY SystemFeature Window
ASUS ROG Aura Listener Sync Proc Invisible Window

MainWindow
Hidden Window
SystemResourceNotifyWindow
peephole_keystone
MediaContextNotificationWindow

ARMOURY GPUPowerSaving Window
UxdService
UxdService
RealtekAudioBackgroundProcessClass
MS_WebcheckMonitor
BluetoothNo

In [2]:
import pyperclip
import re
import pandas as pd

# We'll produce exactly 6 columns:
#  1) Atom
#  2) Shift
#  3) Error
#  4) J Label
#  5) J Value
#  6) error_j

COLUMNS = ["Atom", "Shift", "Error", "J Label", "J Value", "error_j"]


def merge_units(tokens):
    """
    Merge "ppm"/"Hz" with the preceding token.
    E.g. ["0.26", "ppm"] -> ["0.26 ppm"]
         ["7.79", "Hz"]  -> ["7.79 Hz"]
    This helps us remove those units more easily.
    """
    merged = []
    skip_next = False
    for i, t in enumerate(tokens):
        if skip_next:
            skip_next = False
            continue
        if t.lower() in ["ppm", "hz"] and i > 0:
            merged[-1] += " " + t  # combine with previous token
        else:
            merged.append(t)
    return merged


def remove_unit(text, unit):
    """
    Remove trailing 'ppm' or 'Hz' if present.
    e.g. remove_unit("0.26 ppm", "ppm") -> "0.26"
         remove_unit("1.72 Hz", "Hz")   -> "1.72"
    """
    text = text.strip()
    if text.lower().endswith(unit.lower()):
        return text[: -len(unit)].strip()
    return text


def parse_line(line):
    """
    1) Split a line on tabs or 2+ spaces.
    2) Remove empty tokens.
    3) Merge "ppm"/"Hz" with preceding number.
    4) Strip out "ppm"/"Hz" from the resulting tokens.
    """
    parts = re.split(r"\t+|\s{2,}", line.strip())
    parts = [p for p in parts if p]  # remove empty

    parts = merge_units(parts)

    cleaned = []
    for p in parts:
        p = remove_unit(p, "ppm")
        p = remove_unit(p, "hz")
        cleaned.append(p)
    return cleaned


def parse_couplings(tokens):
    """
    Given a list of tokens (like ["J(6,7,8)", "6.62", "1.72", "J(6,7,8-9)", "7.79", "0.73"]),
    split them into triplets: (J Label, J Value, error_j).
    
    If the tokens are not a multiple of 3, the leftover goes into blank fields.
    e.g. ["J(6,7,8)", "6.62"] => J Label="J(6,7,8)", J Value="6.62", error_j=""
    """
    triplets = []
    i = 0
    while i < len(tokens):
        j_label = tokens[i]
        j_value = tokens[i + 1] if (i + 1) < len(tokens) else ""
        j_err   = tokens[i + 2] if (i + 2) < len(tokens) else ""
        triplets.append((j_label, j_value, j_err))
        i += 3
    return triplets

def collect_data():
    # 1) Read raw text from the clipboard (Ctrl+C)
    raw_text = pyperclip.paste()

    # 2) Split into lines & remove lines not part of the table
    lines = raw_text.splitlines()
    cleaned_lines = []
    for ln in lines:
        ln_strip = ln.strip()
        if not ln_strip:
            continue
        # Skip lines like "<sup>..." or any line that starts with "Atom"
        if ln_strip.startswith("<sup>") or ln_strip.lower().startswith("atom"):
            continue
        cleaned_lines.append(ln)

    data = []

    for line in cleaned_lines:
        tokens = parse_line(line)

        # We'll consider a line a "full row" if we have at least 4 tokens:
        #   e.g. ["6,7,8", "CH3", "0.91", "0.26", "J(6,7,8)", "6.62", "1.72"]
        #   => first 2 => Atom, next => Shift, next => Error, leftover => couplings in triplets
        if len(tokens) >= 4:
            # Combine the first 2 tokens into Atom
            atom = tokens[0] + " " + tokens[1]
            shift = tokens[2]
            error = tokens[3]
            leftover = tokens[4:]

            # Parse leftover couplings as triplets (J Label, J Value, error_j)
            if not leftover:
                # If there's no coupling tokens, just put empty J columns
                data.append([atom, shift, error, "", "", ""])
                continue

            triplets = parse_couplings(leftover)

            # The first triplet goes on the same row as Atom/Shift/Error
            j_label, j_value, j_err = triplets[0]
            data.append([atom, shift, error, j_label, j_value, j_err])

            # Additional triplets => new rows, but blank out Atom/Shift/Error
            for trip in triplets[1:]:
                j_label, j_value, j_err = trip
                data.append(["", "", "", j_label, j_value, j_err])

        else:
            # If fewer than 4 tokens, it's a continuation row with couplings only
            # e.g. ["J(6,7,8-9)", "7.79", "0.73"]
            triplets = parse_couplings(tokens)
            for j_label, j_value, j_err in triplets:
                data.append(["", "", "", j_label, j_value, j_err])
        
    return pd.DataFrame(data, columns=COLUMNS)
    


In [15]:
import pyautogui
import csv

def record_click_positions(num_positions, output_csv="positions.csv"):
    """
    Records 'num_positions' mouse coordinates and saves them to 'output_csv'.
    """
    positions = []

    for i in range(num_positions):
        print(f"\nHover the mouse over position {i+1} and press ENTER in this console...")
        input("Press ENTER to record this position.")
        x, y = pyautogui.position()
        positions.append((x, y))
        print(f"Position {i+1} recorded: ({x}, {y})")

    # Save all positions to CSV
    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["PositionIndex", "X", "Y"])
        for i, (x, y) in enumerate(positions, start=1):
            writer.writerow([i, x, y])

    print(f"\nSaved {num_positions} positions to '{output_csv}'.")

def main():
    # Ask user how many positions to record
    num_clicks_str = input("How many click positions do you want to record? ")
    try:
        num_clicks = int(num_clicks_str)
    except ValueError:
        print("Invalid number. Exiting.")
        return

    record_click_positions(num_clicks)

if __name__ == "__main__":
    main()



Hover the mouse over position 1 and press ENTER in this console...
Position 1 recorded: (1378, 204)

Hover the mouse over position 2 and press ENTER in this console...
Position 2 recorded: (430, 156)

Hover the mouse over position 3 and press ENTER in this console...
Position 3 recorded: (1076, 559)

Saved 3 positions to 'positions.csv'.


In [ ]:
import os
import time
import pyautogui
import pandas as pd
from pywinauto import Application, Desktop
import pyperclip

def get_mestrenova():
    try:
        app = Application(backend="uia").connect(title_re="^MestReNova$")
        window = app.top_window()
        window.set_focus()
        print("MestReNova is now active.")
    except Exception as e:
        print("Error: MestReNova window not found.", e)
    return 0

MESTRE_EXE = r"C:\Program Files\Mestrelab Research S.L\MestReNova\MestReNova.exe"

# 2) Folder containing SDF files:
SDF_FOLDER = r"C:\Users\bikas\Documents\test"

# 3) CSV file to store results:
output_folder = r"C:\Users\bikas\Documents\c7_raw"

# 4) Window title pattern. Adjust after you run the "list all windows" snippet:
WINDOW_TITLE_REGEX = ".*Mnova.*"  # or ".*MestReNova.*" or something else
sleep_time = 1
def start_or_connect_mestre():
    """Start MestReNova if not running, or connect to it if already running."""
    # Try connecting if it is already open:
    try:
        app = Application(backend="uia").connect(title_re=WINDOW_TITLE_REGEX)
        print("MestReNova found, connecting...")
    except:
        print("MestReNova not found, starting it...")
        # Start the application:
        app = Application(backend="uia").start(MESTRE_EXE)
        # Optional: Wait for CPU to drop, meaning the app likely finished loading
        app.wait_cpu_usage_lower(threshold=5, timeout=30)
    
    # At this point, we are sure the app is running.
    # Let's get the main window:
    main_win = app.window(title_re=WINDOW_TITLE_REGEX)
    # Wait for the window to be "ready":
    main_win.wait("ready", timeout=30)
    main_win.set_focus()
    time.sleep(sleep_time)
    print("MestReNova is active and focused now.")
    return app, main_win

def open_sdf_file(file_path):
    """Send Ctrl+O, type path, press Enter."""
    pyautogui.hotkey("ctrl", "o")  # Open file dialog
    time.sleep(sleep_time)
    pyautogui.write(file_path)
    pyautogui.press("enter")
    # Wait a bit for the file to load:
    time.sleep(3*sleep_time)

def close_sdf_file(x1_close, y1_close, x2_close, y2_close):
    """Close the SDF file."""
    pyautogui.click(x=x1_close, y=y1_close)
    pyautogui.click(x=x2_close, y=y2_close)
    #pyautogui.click(x=x3_close, y=y3_close)


def predict_nmr(x1, y1, x_pred, y_pred, x_copy, y_copy):
    """Click on 1H Prediction tab, then Table View, then copy the data."""
    pyautogui.click(x=x1, y=y1)
    time.sleep(sleep_time)
    pyautogui.click(x=x_pred, y=y_pred)
    time.sleep(sleep_time)
    #pyautogui.click(x=x_table, y=y_table)
    #time.sleep(sleep_time)
    pyautogui.click(x=x_copy, y=y_copy)
    time.sleep(sleep_time)  # Ensure clipboard data is ready
    a = collect_data()
    
    '''Click on 13C Prediction tab, then copy the data'''
    pyautogui.click(x=130, y=95)
    time.sleep(sleep_time)
    pyautogui.click(x=1390, y=413)
    time.sleep(sleep_time)
    b = collect_data()
    return pd.concat([a, b], axis=1)

#pd.read_clipboard()
#pyperclip.paste()

# -----------------------------
# MAIN AUTOMATION LOGIC
# -----------------------------
def main():
    # 1) Start or connect to MestReNova:
    get_mestrenova()

    # 2) Get coordinates for the 1H Prediction and Table View buttons:
    #    (We do this once, then re-use them for each file.)
    x1, y1 = 307, 49
    x_pred, y_pred, x_copy, y_copy = 33, 95, 196, 370
    x1_close, y1_close, x2_close, y2_close= 452, 189, 1044, 534 
    # 3) Process each SDF file:
    for file_name in os.listdir(SDF_FOLDER):
        if file_name.lower().endswith(".sdf"):
            file_path = os.path.join(SDF_FOLDER, file_name)
            
            print(f"\n=== Processing: {file_name} ===")
            open_sdf_file(file_path)
            
            # Perform 1H NMR Prediction:
            predicted_data = predict_nmr(x1, y1, x_pred, y_pred, x_copy, y_copy)
            close_sdf_file(x1_close, y1_close, x2_close, y2_close)
            # Save the csv with the input name
            name = file_name.replace('.sdf', '')
            output = os.path.join(output_folder, name)
            predicted_data.to_csv(output + ".csv", index=False)

    print("\nAll files processed successfully!")

if __name__ == "__main__":
    main()


MestReNova is now active.

=== Processing: imp_dsgdb9nsd_000012.nmredata.sdf.sdf ===

=== Processing: imp_dsgdb9nsd_000013.nmredata.sdf.sdf ===

All files processed successfully!
